# Data Programming cw 2 

#### contents
- 1
- 2
- 3
etc...

Imports we will need

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from webdriver_manager.chrome import ChromeDriverManager
import time

we need to gather some more data.
This time we maximise the window so more content can be loaded and scraped faster. 

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
aging_neuroscience = "https://www.frontiersin.org/journals/aging-neuroscience/articles"
synaptic_neuroscience = "https://www.frontiersin.org/journals/synaptic-neuroscience/articles"
driver.get(synaptic_neuroscience)
driver.maximize_window()  # make window bigger so more content can be loaded at once for faster scraping.

Now check that the page has loaded and the cookies have been accepted.

In [3]:
wait = WebDriverWait(driver, 10)
cookies = wait.until(expected_conditions.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
cookies.click()

For some reason the old method od scrolling doesnt seem to work on these pages. To overcome this we check the height of the page at each iteration of a loop and scroll by that amount. The added sleep command gives the page time to load and means the height changes on every iteration of the loop so it can be checked. It is not a very elegant or robust solution but it works. 

In [4]:
while True:
    previous_height = driver.execute_script("return document.documentElement.scrollHeight")
    driver.execute_script("window.scrollTo(0, " + str(previous_height) + ");")
    time.sleep(1)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if previous_height == new_height:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [15]:
def get_article_cards(soup):
    scraped_article_cards = soup.find_all('article', class_='CardArticle')
    article_cards = []
    for article_card in scraped_article_cards:
        article_cards.append(article_card)
    return article_cards


article_cards = get_article_cards(soup)


In [27]:
print(article_cards[6])

<article class="CardArticle CardArticle--hasImg CardArticle--hasMetrics"><a class="CardArticle__wrapper" data-event="cardArticle-a-103389/fnsyn20221023256" href="https://www.frontiersin.org/articles/10.3389/fnsyn.2022.1023256"><div class="CardArticle__info"><div class="CardArticle__header"><p class="CardArticle__type">Mini Review</p> <p class="CardArticle__date">
          Published on 05 Dec 2022
        </p></div> <h1 class="CardArticle__title">Diverse organization of voltage-gated calcium channels at presynaptic active zones</h1> <!-- --> <ul class="CardArticle__authors"><li>Weijia Zhang</li><li>He-Hai Jiang</li><li>Fujun Luo</li></ul> <div class="CardArticle__spacer"></div> <figure class="CardArticle__mask"><img alt="Diverse organization of voltage-gated calcium channels at presynaptic active zones" class="CardArticle__mask__img" src="https://www.frontiersin.org/files/myhome article library/1023256/1023256_Thumb_400.jpg"/></figure> <div class="CardArticle__journal"><div class="Card

In [24]:
#article_type = article.find('p', class_='CardArticle__type')
#article_date = article.find('p', class_='CardArticle__date')

def get_article_data(article_cards):
    article_data = []
    for article in article_cards:
        article_title = article.find('h1', class_='CardArticle__title')
        article_data.append(article_title.text)
    return article_data

In [26]:
article_data = get_article_data(article_cards)

print(article_data[1])

GRIN2B-related Neurodevelopmental Disorder: Current Understanding of Pathophysiological Mechanisms
